In [ ]:
# don't know why but need an update
!sudo apt update
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
# install findspark 
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [96.1 kB]
Get:12 http://ppa.launchp

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

import findspark
findspark.init()

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"andreaadami","key":"24de4196d6cdef6c1a7c6e7ec023b6a3"}'}

In [ ]:
# run this code in order to provide the API and connect to Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# this command is used to access "census/2013..." and download the zip file
!kaggle datasets download -d census/2013-american-community-survey

100% 912M/916M [00:21<00:00, 41.8MB/s]
100% 916M/916M [00:21<00:00, 44.6MB/s]


In [ ]:
# now that everything is set, we need to unzip the file and load in the notebook 
import zipfile
zip_ref = zipfile.ZipFile('2013-american-community-survey.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas as pd
import librosa
import statistics 
import glob
import plotly.express as px
from pathlib import Path
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler  
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import PCA
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "2gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [ ]:
data_a = spark.read.csv('/content/files/ss13pusa.csv',header=True, inferSchema=True)#load the data and get a row (this is a dataframe)
data_b = spark.read.csv('/content/files/ss13pusb.csv',header=True, inferSchema=True) 

data = data_a.union(data_b)

data.show(2)

+---+--------+-------+----+---+-------+-----+----+---+-----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+----+----+---+-----+-----+-----+-----+------+---+---+----+----+-----+----+-----+----+----+----+----+----+----+----+----+----+---+---+----+----+---+----+----+----+---+----+---+-----+----+---+----+---+----+---+-----+-----+------+---+-------+----+---+-----+-----+-----+----+----+----+----+----+-------+-----+---+------+--------+----+---+----+----+-----+-----+----+------+-------+-----+-------+------+------+-----+-----+-----+-------+------+------+-----+------+-----+------+------+---+-------+---------+----+----+------+----+----+-----+-----+-----+------+-----+------+------+------+-----+------+------+------+-------+------+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----+-----+-----+-----+------+------+------+-----+------+-------+--

In [ ]:
#transforming all the dataframe columns to int for correctly fill Nan and null value

from pyspark.sql.types import IntegerType
for col in data.columns:
  data = data.withColumn(col,data[col].cast(IntegerType()))

data = data.na.fill(0) 

In [ ]:
#removing the PERNP column
new_col = data.columns
new_col.remove('PERNP')
new_col.remove('PINCP')




vectorAssembler = VectorAssembler(inputCols = new_col,    
                                  outputCol = 'features') 


df_full = vectorAssembler.transform(data)
df_full = df_full.select(['features', 'PINCP'])


df_full.show(5)

+--------------------+-----+
|            features|PINCP|
+--------------------+-----+
|(281,[1,2,3,4,5,6...|    0|
|(281,[1,2,3,4,5,6...|52000|
|(281,[1,2,3,4,5,6...|99000|
|(281,[1,2,3,4,5,6...|    0|
|(281,[1,2,3,4,5,6...|    0|
+--------------------+-----+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

pca = PCA(k=20, 
          inputCol="features", 
          outputCol="pcaFeatures")

model = pca.fit(df_full)

result = model.transform(df_full)
result.show(5)

+--------------------+-----+--------------------+
|            features|PINCP|         pcaFeatures|
+--------------------+-----+--------------------+
|(281,[1,2,3,4,5,6...|    0|[-84.065349464680...|
|(281,[1,2,3,4,5,6...|52000|[-239.98024490026...|
|(281,[1,2,3,4,5,6...|99000|[-260.53340347800...|
|(281,[1,2,3,4,5,6...|    0|[-154.06172172305...|
|(281,[1,2,3,4,5,6...|    0|[-154.06242385407...|
+--------------------+-----+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import MaxAbsScaler
from pyspark.sql.functions import col
# train test split 
split_df_pp = result.randomSplit([0.75, 0.25])
df_train = split_df_pp[0]
df_test = split_df_pp[1]

#here we can put a good condition on PINCP > 0 


##Scaling DenseVector (input)

#maxAbs scaler
scaler = MaxAbsScaler(inputCol="pcaFeatures", outputCol="scaled_pca_Features")
scalerModel = scaler.fit(df_train)
#scaled train
df_pp_train = scalerModel.transform(df_train)
#scaled test
df_pp_test = scalerModel.transform(df_test) #it is transformed in accordance with range of train



##Scaling Target variable (PINCP)

min_ = df_pp_train.agg({"PINCP": "min"}).collect()[0][0]
max_ = df_pp_train.agg({"PINCP": "max"}).collect()[0][0]

#scaled train
df_pp_train = df_pp_train.withColumn('scaled_PINCP', (col('PINCP') - min_ )/ max_)
#scaled test
df_pp_test = df_pp_test.withColumn('scaled_PINCP', (col('PINCP')-min_ )/ max_) #it is transformed in accordance with range of train


df_pp_train.show(2)

+--------------------+-----+--------------------+--------------------+-------------------+
|            features|PINCP|         pcaFeatures| scaled_pca_Features|       scaled_PINCP|
+--------------------+-----+--------------------+--------------------+-------------------+
|(281,[1,2,3,4,5,6...|76400|[-1346063.6211065...|[-0.9015539176260...|0.07053291536050156|
|(281,[1,2,3,4,5,6...|45880|[-577225.43648477...|[-0.3866086605835...|0.04661442006269592|
+--------------------+-----+--------------------+--------------------+-------------------+
only showing top 2 rows



In [ ]:
## boost it! scaled
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(featuresCol = 'scaled_pca_Features', 
                   labelCol = 'scaled_PINCP',
                   maxIter=10,
                   #maxDepth = ,
                   #stepSize = ,
                   seed = 123,
                   )

#fitting the model on train data
gbt_model = gbt.fit(df_pp_train)

#prediction on test data
gbt_predictions = gbt_model.transform(df_pp_test)

gbt_predictions.select('prediction','scaled_PINCP', 'PINCP', 'features').show(5)

+-------------------+-------------------+------+--------------------+
|         prediction|       scaled_PINCP| PINCP|            features|
+-------------------+-------------------+------+--------------------+
|0.03917944637495162|0.05550314465408805| 57000|(281,[1,2,3,4,5,6...|
|0.16768133677676733|0.20605345911949685|248500|(281,[1,2,3,4,5,6...|
|0.11596047294714994|0.12067610062893082|139900|(281,[1,2,3,4,5,6...|
|0.08335437162907182|0.10015723270440252|113800|(281,[1,2,3,4,5,6...|
|0.11568238627135882|0.08679245283018867| 96800|(281,[1,2,3,4,5,6...|
+-------------------+-------------------+------+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
gbt_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="r2")
r2 = gbt_evaluator.evaluate(gbt_predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("R Squared (R2) on test data = %g" % r2)

Root Mean Squared Error (RMSE) on test data = 0.0101282
R Squared (R2) on test data = 0.935693


In [ ]:
# https://spark.apache.org/docs/latest/ml-classification-regression.html
# https://spark.apache.org/docs/latest/mllib-linear-methods.html

In [ ]:
from pyspark.ml.regression import LinearRegression

#define the model
lr_ridge = LinearRegression(featuresCol = 'scaled_pca_Features', 
                            labelCol = 'scaled_PINCP',
                            maxIter=2,
                            regParam=0.8, 
                            elasticNetParam=0) #lasso condition (alpha = 0)

# Fit the model
lr_ridge_model = lr_ridge.fit(df_pp_train)

# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lr_ridge_model.coefficients))
print("Intercept: %s" % str(lr_ridge_model.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lr_ridge_model.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [4.1606986510062495e-07,0.0010195423040238586,-0.0004431341433859792,0.024654530399565774,0.020233684037871954,0.009662505345729898,0.00011822581293560417,0.007467572422878289,-0.0017117885785917688,-0.00021453347978572936,-0.0027291229990445026,-7.383865081190122e-05,6.399079936360878e-05,-3.88120915809985e-05,-0.0011515043440148548,-3.107482610126811e-06,-2.05682531449725e-05,-2.8393335086804862e-05,-1.999962602359967e-05,-6.271668683527372e-06]
Intercept: 0.03252665018992613
numIterations: 1
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
|0.034910237447102394|
|0.012207788265207908|
|0.014530434972803034|
|0.002628050193462...|
| 0.01118218696607684|
|0.015123985536569219|
|-0.00511170142690609|
|-0.02191718131262...|
| 0.02048548948656418|
|-0.00904884033287...|
|-0.00867480487506...|
| 0.04495535676232316|
| 0.16292985850655606|
|0.022724254306736295|
| 0.08214149809970944|
| 0.04539847170278605|
| 0.06901729297268024|
| 

In [ ]:
#prediction on test data
ridge_predictions = lr_ridge_model.transform(df_pp_test)


ridge_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="rmse")
rmse_ridge = ridge_evaluator.evaluate(ridge_predictions)

ridge_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="r2")
r2_ridge = ridge_evaluator.evaluate(ridge_predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_ridge)
print("R Squared (R2) on test data = %g" % r2_ridge)

Root Mean Squared Error (RMSE) on test data = 0.0377814
R Squared (R2) on test data = 0.0921576


In [ ]:
### modelling with trainvalidation split (Linear regression)
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

##here train test split

lin_reg = LinearRegression(featuresCol = 'scaled_pca_Features',
                           labelCol = 'scaled_PINCP',
                           maxIter=2)

# grid of parameters to use (fitting all the combination of models) 
param_Grid = ParamGridBuilder()\
    .addGrid(lin_reg.regParam, [0.2, 0.1, 0.01])\
    .addGrid(lin_reg.fitIntercept, [False, True])\
    .addGrid(lin_reg.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()


# train-validation settings for evaluation
train_val_split = TrainValidationSplit(estimator = lin_reg, #model 
                                       estimatorParamMaps = param_Grid, #grid of parameters
                                       evaluator = RegressionEvaluator(labelCol ='scaled_PINCP',predictionCol = 'prediction'), #evaluator function
                                       trainRatio = 0.8 ) #train validation split          


# fitting the model using the best combination of parameters 
Lin_reg_model = train_val_split.fit(df_pp_train)

# prediction on test data using the best fitted model
Lin_reg_model.transform(df_pp_test)\
    .select('scaled_pca_Features', 'scaled_PINCP', 'prediction')\
    .show()

+--------------------+--------------------+--------------------+
| scaled_pca_Features|        scaled_PINCP|          prediction|
+--------------------+--------------------+--------------------+
|[-0.8787471152422...| 0.04905956112852665|0.046019226567717834|
|[-0.0045907244093...| 0.03652037617554859|0.036005108831260235|
|[-0.0229210835566...| 0.08111285266457681| 0.07160400809007596|
|[-0.0980039737330...| 0.08158307210031349| 0.07201421287413995|
|[-0.4155844841892...| 0.07452978056426332| 0.06635936160915688|
|[-0.4416218787598...| 0.06865203761755485| 0.06171130261529782|
|[-0.5668504395248...|0.024764890282131663|0.026599470754072374|
|[-0.4697174443819...| 0.13197492163009406| 0.11222714738715922|
|[-0.0442745810572...|0.014968652037617556| 0.01879291530713562|
|[-0.2787244443170...|0.014968652037617556|0.018792874645050105|
|[-0.4889248325296...|0.026332288401253918|0.027865022924304053|
|[-0.3560696750987...| 0.09686520376175549| 0.08417580090222539|
|[-0.0722094693287...|0.0

In [ ]:
#prediction on test data

lin_reg_pred = Lin_reg_model.transform(df_pp_test)

lin_reg_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="rmse")
rmse_linreg = lin_reg_evaluator.evaluate(lin_reg_pred)

lin_reg_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="r2")
r2_linreg = lin_reg_evaluator.evaluate(lin_reg_pred)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_linreg)
print("R Squared (R2) on test data = %g" % r2_linreg)

Root Mean Squared Error (RMSE) on test data = 0.00804295
R Squared (R2) on test data = 0.959461


In [ ]:
#prediction on train data

lin_reg_pred_t = Lin_reg_model.transform(df_pp_train)

lin_reg_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="rmse")
rmse_linreg_t = lin_reg_evaluator.evaluate(lin_reg_pred_t)

lin_reg_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="r2")
r2_linreg_t = lin_reg_evaluator.evaluate(lin_reg_pred_t)

print("Root Mean Squared Error (RMSE) on train data = %g" % rmse_linreg_t)
print("R Squared (R2) on train data = %g" % r2_linreg_t)

Root Mean Squared Error (RMSE) on train data = 0.00799951
R Squared (R2) on train data = 0.959459


In [ ]:
## boost it! scaled
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(featuresCol = 'scaled_pca_Features',
                   labelCol = 'scaled_PINCP',
                   maxIter=3,
                   seed = 123)

# grid of parameters to use (fitting all the combination of models) 
param_Grid_gbt = ParamGridBuilder()\
    .addGrid(gbt.maxDepth, [2, 3, 5])\
    .addGrid(gbt.maxBins, [28 , 32])\
    .addGrid(gbt.stepSize, [0.01 , 0.05, 0.1])\
    .build()


# train-validation settings for evaluation
train_val_split_gbt = TrainValidationSplit(estimator = gbt, #model 
                                       estimatorParamMaps = param_Grid_gbt, #grid of parameters
                                       evaluator = RegressionEvaluator(labelCol ='scaled_PINCP',predictionCol = 'prediction'), #evaluator function
                                       trainRatio = 0.8 ) #train validation split          




#fitting the model on train data
gbt_model = train_val_split_gbt.fit(df_pp_train)

#prediction on test data
gbt_predictions = gbt_model.transform(df_pp_test)

gbt_predictions.select('prediction','scaled_PINCP', 'PINCP', 'features').show(5)

+--------------------+-------------------+-----+--------------------+
|          prediction|       scaled_PINCP|PINCP|            features|
+--------------------+-------------------+-----+--------------------+
|0.028019379152082628|0.04905956112852665|49000|(281,[1,2,3,4,5,6...|
|0.029054259629427783|0.03652037617554859|33000|(281,[1,2,3,4,5,6...|
| 0.07740122372304847|0.08111285266457681|89900|(281,[1,2,3,4,5,6...|
|  0.0861178555733775|0.08158307210031349|90500|(281,[1,2,3,4,5,6...|
| 0.06488609699620822|0.07452978056426332|81500|(281,[1,2,3,4,5,6...|
+--------------------+-------------------+-----+--------------------+
only showing top 5 rows



In [ ]:
gbt_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="r2")
r2 = gbt_evaluator.evaluate(gbt_predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("R Squared (R2) on test data = %g" % r2)

Root Mean Squared Error (RMSE) on test data = 0.0125938
R Squared (R2) on test data = 0.900607


In [ ]:
##score on train
gbt_predictions_train = gbt_model.transform(df_pp_train)

gbt_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="rmse")
rmse_t = gbt_evaluator.evaluate(gbt_predictions_train)

gbt_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="r2")
r2_t = gbt_evaluator.evaluate(gbt_predictions_train)

print("Root Mean Squared Error (RMSE) on train data = %g" % rmse_t)
print("R Squared (R2) on train data = %g" % r2_t)


Root Mean Squared Error (RMSE) on test data = 0.0124974
R Squared (R2) on test data = 0.901053
